In [6]:
username = "admin"
password = "thebridge2022"
host = "tweet-search-2.czjoi0srhr5i.eu-west-3.rds.amazonaws.com" #"database-2.crerrxwepn0b.us-east-2.rds.amazonaws.com"
port = 3306

### Conexion BD

In [7]:
import pymysql

In [8]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [9]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.28'}


### Creación de DB

In [10]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
create_db = '''CREATE DATABASE tweet_search'''
cursor.execute(create_db)

# # Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor.execute(drop_db)

# # La volvemos a crear
# cursor.execute(create_db)
# # el output es el numero de filas afectadas

1

In [11]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'},
 {'Database': 'tweet_search'}]

### Creación de tablas

In [14]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''


# crear una tabla que se llame tweet_data
create_table = '''
CREATE TABLE tweet_data (
id INT,
text TEXT,
created_at DATE,
author_id INT,
primary key (author_id)
)
'''

cursor.execute(create_table)

0

In [17]:
create_table = '''
CREATE TABLE user_data (
author_id INT,
user_id INT,
user_name TEXT,
retweet_count INT,
reply_count INT,
like_count INT,
quote_count INT,
primary key (author_id)
)
'''

cursor.execute(create_table)

0

### Seleccionar la BD

In [13]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE tweet_search'''
cursor.execute(use_db)

0

In [18]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_tweet_search': 'tweet_data'},
 {'Tables_in_tweet_search': 'user_data'}]

In [23]:

# # Si queremos borrar alguna tabla

# drop_table = '''DROP TABLE tweet_search'''
# cursor.execute(drop_table)


0

### Insertar datos de un CSV

In [22]:
import pandas as pd

In [ ]:
# df = pd.DataFrame({
#     'name': ['France', 'Japan', 'Argentina', 'Brasil'],
#     'continent' : ['Europe', 'Asia', 'America', 'America'],
#     'population': [70, 126, 44, 209],
#     'gdp': [2.7, 4.9, 0.5, 1.8]
# })

# df.head()

In [ ]:
# df.to_csv('data/country_data.csv', sep=';')

In [66]:
df = pd.read_csv('data/tweet_search.csv', sep=';')

In [68]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [69]:
df.columns

Index(['username', 'user_id', 'tweet_text', 'tweet_id', 'author_id',
       'created_at', 'retweet_count', 'reply_count', 'like_count',
       'quote_count'],
      dtype='object')

In [74]:
tweet_data = df[['author_id', 'tweet_id', 'tweet_text', 'created_at', 'retweet_count', 'reply_count', 'like_count', 'quote_count']]

In [75]:
tweet_data.head()

,author_id,tweet_id,tweet_text,created_at,retweet_count,reply_count,like_count,quote_count
0,1162694149956603904,1526946552492789760,Esta es la historia de Isis🤗 que decidió hacer...,2022-05-18 15:23:22+00:00,0,0,0,0
1,1162694149956603904,1526886949839065088,Gracias @victorianoI y equipo por el análisis ...,2022-05-18 11:26:31+00:00,0,0,0,0
2,1336771626306854912,1526842834741039105,¿Cómo combatir el paro juvenil🥵? #tipdeldia En...,2022-05-18 08:31:13+00:00,4,1,8,0
3,114739070,1526479745185198080,Los perfiles tecnológicos más demandados en el...,2022-05-17 08:28:26+00:00,2,0,1,0
4,15993641,1526133158743359488,La 2da presentación-coloquio del #LibroIC en #...,2022-05-16 09:31:13+00:00,4,0,4,0


In [76]:
user_data = df[['author_id', 'user_id', 'username']]

In [77]:
user_data.head()

,author_id,user_id,username
0,1162694149956603904,1162694149956603904,TheBridge_Tech
1,1162694149956603904,10977452,victorianoi
2,1336771626306854912,1162694149956603904,TheBridge_Tech
3,114739070,1162694149956603904,TheBridge_Tech
4,15993641,1162694149956603904,TheBridge_Tech


In [78]:
print(username)
print(password)  
print(host)
print('tweet_search')

admin
thebridge2022
tweet-search-2.czjoi0srhr5i.eu-west-3.rds.amazonaws.com
tweet_search


In [27]:
from sqlalchemy import create_engine

# create sqlalchemy engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'tweet_search'))
# engine = create_engine("mysql+pymysql://my_user:my_password@my_host/my_database")

In [31]:
# insertamos todo el dataframe
df.to_sql(name='tweet_search', con=engine, if_exists= 'append', index=False)


In [ ]:
db.commit()

In [33]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM tweet_search'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

OperationalError: (1412, 'Table definition has changed, please retry transaction')

In [ ]:
mi_tabla[4]['population']

209

In [34]:
pd.DataFrame(mi_tabla)

""


### Leer datos

In [35]:
sql = '''SELECT * FROM tweet_search'''
cursor.execute(sql)

OperationalError: (1412, 'Table definition has changed, please retry transaction')

In [27]:
mi_lista = cursor.fetchall()
len(mi_lista)

1

In [28]:
mi_lista

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.74}]

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [50]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [19]:
# sql = '''SELECT * FROM countryy'''
# cursor.execute(sql)
# print(cursor.fetchall())
# print('Proceso finalizado')

In [20]:
try:
    sql = '''SELECT * FROM tweet_search'''
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)

print('Proceso finalizado')

(1146, "Table 'tweet_search.tweet_search' doesn't exist")
Proceso finalizado


### Cerrar la conexión cuando acabemos

In [92]:
db.close()